In [19]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

import gymnasium as gym
from gymnasium.wrappers import RescaleAction, NormalizeObservation, NormalizeReward
import nav2d       
import numpy as np
import os
from tqdm import tqdm

test a given model:

In [20]:
import pyautogui
    
# paths:
results_path = "results/Nav2D_SAC_SB3_results"
desired_result_path = "result_004"
run = "run_100"
normalize = True

# scales:
reward_scale = {"rew_head_scale" : 10.0,
                "rew_head_approach_scale" : 220.0,
                "rew_dist_scale" : 3.5,
                "rew_dist_approach_scale" : 200.0,
                "rew_goal_scale" : 5000.0,
                "rew_obst_scale" : -1000.0, 
                "rew_time" : -0.25}

# visualize:
testing_length = 50

# render settings:
width = 1280
height = 1280
default_camera_config = {"azimuth" : 90.0, "elevation" : -90.0, "distance" : 3, "lookat" : [0.0, 0.0, 0.0]}
camera_id = 2

DEFAULT_CAMERA = "overhead_camera"
ENABLE_FRAME = True
RENDER_EVERY_FRAME = True

# make a single environment:
env = gym.make("Nav2D-v0",
            render_mode = "human",
            width = width, 
            height = height,
            default_camera_config = default_camera_config, 
            camera_id = camera_id, 
            max_episode_steps = 1000, 
            is_eval = False,
            reward_scale_options = reward_scale)

# if vectorize is True:
if normalize:
    # make a vector consisting of a single environment:
    eval_env = DummyVecEnv([lambda: env])

    # load the normalization stats:
    eval_env = VecNormalize.load(os.path.join(results_path, desired_result_path, "vec_norm_env_stats.pkl"), eval_env)
    eval_env.training = False
    eval_env.norm_reward = False
    print("created a normalized environment!")

    # load model:
    model = SAC.load(os.path.join(results_path, desired_result_path, run), env = eval_env)
else:
    # load model as is:
    model = SAC.load(os.path.join(results_path, desired_result_path, run))

    # set environment:
    eval_env = env
    print("created a base environment!")

created a normalized environment!


visualize the performance of the model:

In [ ]:
# for every test episode:
for eps in range(testing_length):
    if normalize:
        obs = eval_env.reset()
    else:
        obs, _ = eval_env.reset()

    done = False

    # while not done:
    while not done:
        action, _ = model.predict(obs, deterministic = True)
        if normalize:
            nobs, reward, term, _ = eval_env.step(action)
            done = term
        else:
            nobs, reward, term, trunc, _ = eval_env.step(action)
            done = term or trunc

        # advance observation, reset if not:
        obs = nobs if not done else eval_env.reset()
    
        # render for user:
        eval_env.render()
 
# close when done:
eval_env.close()